## Understanding LangChain

main source: https://www.youtube.com/watch?v=nAmC7SoVLd8&list=PLeo1K3hjS3uu0N_0W6giDXzZIcB07Ng_F&index=1&ab_channel=codebasics


## Install Dependancies 

In [1]:
%pip install langchain
%pip install OpenAI
%pip install -U langchain-openai 
%pip install wikipedia
%pip install numexpr



Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## LLMs and Prompts

In [ ]:
from langchain_openai import OpenAI
import os
openai_api_key = ''
os.environ['OPENAI_API_KEY'] = openai_api_key
llms = OpenAI(temperature= 0.6)

In [ ]:
name = llms('How can I use db FROM SNOWFLAKE HERE ')
print(name)

In [4]:
from langchain.prompts import PromptTemplate

prompot_template_name = PromptTemplate(
    input_variables= ['cuisine'],
    template= "I want to open a restaurant for {cuisine} food. Suggest a fancy name for this"
)

prompot_template_name.format(cuisine = "Italian")

'I want to open a restaurant for Italian food. Suggest a fancy name for this'

In [8]:
from langchain.chains import LLMChain

chain = LLMChain(llm = llms, prompt = prompot_template_name)
chain.run("Mexican")

c:\Python312\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
c:\Python312\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'\n\n"El Dorado Cantina"'

## Chains

### Trying Simple Sequential Chain

In [13]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain


prompot_template_name = PromptTemplate(
    input_variables= ['cuisine'],
    template= "I want to open a restaurant for {cuisine} food. Suggest a fancy name for this"
)

name_chain = LLMChain(llm= llms , prompt = prompot_template_name)

prompot_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template = """Suggest some menu items for {restaurant_name}. Return it as a comma separated list not list of numbers"""
)

food_items_chain = LLMChain(llm = llms , prompt = prompot_template_items)

In [17]:
from langchain.chains import SimpleSequentialChain

chain = SimpleSequentialChain(chains = [name_chain , food_items_chain])
response = chain.run("Turkish")
print(response)




1. Meze Platter (Hummus, Baba Ghanoush, Dolma, Tabouleh)
2. Turkish Kebabs (Lamb, Chicken, or Vegetarian)
3. Adana Kofte (Spicy Ground Beef Kebabs)
4. Lahmacun (Turkish Pizza)
5. Iskender Kebab (Sliced Lamb with Tomato Sauce and Yogurt)
6. Manti (Turkish Dumplings)
7. Imam Bayildi (Stuffed Eggplant)
8. Borek (Savory Pastry filled with Cheese or Meat)
9. Pide (Turkish Flatbread with various toppings)
10. Baklava (Sweet Pastry with Nuts and Honey)


**Note:** it gives me just one result for one prompt

### Sequential Chain

In [32]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain


prompot_template_name = PromptTemplate(
    input_variables= ['cuisine'],
    template= "I want to open a restaurant for {cuisine} food. Suggest a fancy name for this"
)

name_chain_2 = LLMChain(llm= llms , prompt = prompot_template_name,output_key="restaurant_name")

prompot_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template = """Suggest some menu items for {restaurant_name}."""
)

food_items_chain_2 = LLMChain(llm = llms , prompt = prompot_template_items , output_key="menu_items")

In [34]:
from langchain.chains import SequentialChain 

sequential_chain = SequentialChain(
    chains=[name_chain_2, food_items_chain_2],
    input_variables=['cuisine'],
    output_variables=['restaurant_name', 'menu_items']
)

output = sequential_chain({'cuisine': 'Turkish'})
print(output)

{'cuisine': 'Turkish', 'restaurant_name': '\n\n"Anatolian Delights"', 'menu_items': '\n\n1. Lamb Kebabs: Tender pieces of marinated lamb grilled to perfection, served with rice and a side of tzatziki sauce.\n\n2. Dolma: Grape leaves stuffed with a flavorful mixture of rice, herbs, and spices, served with a lemon yogurt sauce.\n\n3. Adana Kofte: Spicy ground beef and lamb meatballs, seasoned with traditional Anatolian spices and grilled to perfection.\n\n4. Meze Platter: A selection of small dishes including hummus, baba ghanoush, tabbouleh, and stuffed grape leaves, served with warm pita bread.\n\n5. Lahmacun: Turkish-style flatbread topped with a savory mixture of ground lamb, tomatoes, and spices.\n\n6. Iskender Kebab: Slices of tender beef or lamb served over a bed of pita bread, topped with a rich tomato and yogurt sauce.\n\n7. Manti: Small dumplings filled with ground meat and served with a garlic yogurt sauce and a sprinkle of sumac.\n\n8. Kebab Salad: Grilled chicken or beef ske

## Agents

In [45]:
from langchain.agents import AgentType , initialize_agent, load_tools
from langchain.llms import OpenAI

tools = load_tools(["wikipedia","llm-math"], llm=llms)

agent = initialize_agent(
    tools,
    llms,
    agent =AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

agent.run("When was Elon musk born? and What is his age right now in 2024?")



> Entering new AgentExecutor chain...
 I should use wikipedia to find information about Elon Musk's birthdate and current age.
Action: wikipedia
Action Input: "Elon Musk"
Observation: Page: Elon Musk
Summary: Elon Reeve Musk ( EE-lon; born June 28, 1971) is a businessman and investor. He is the founder, chairman, CEO, and CTO of SpaceX; angel investor, CEO, product architect, and former chairman of Tesla, Inc.; owner, executive chairman, and CTO of X Corp.; founder of the Boring Company and xAI; co-founder of Neuralink and OpenAI; and president of the Musk Foundation. He is one of the wealthiest people in the world; as of April 2024, Forbes estimates his net worth to be $178 billion. 
A member of the wealthy South African Musk family, Musk was born in Pretoria and briefly attended the University of Pretoria before immigrating to Canada at age 18, acquiring citizenship through his Canadian-born mother. Two years later, he matriculated at Queen's University at Kingston in Canada. Musk 

'Elon Musk will be 53 years old in 2024.'

## Memory

### Before adding memory

In [47]:
chain = LLMChain(llm=llms , prompt = prompot_template_name)
name = chain.run("Mexico")
print(name)



"El Sabor de México" (The Flavor of Mexico)


In [48]:
name = chain.run("Indian")
print(name)



"Maharaja's Palace: An Exquisite Indian Dining Experience"


In [49]:
type(chain.memory)

NoneType

### After adding memory

In [50]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()

chain = LLMChain(llm = llms, prompt=prompot_template_name, memory=memory )
name = chain.run("Mexican")
print(name)



"El Dorado Cantina"


In [51]:
name = chain.run("Indian")
print(name)



"Maharaja's Palace: Fine Indian Cuisine"


In [52]:
chain.memory

ConversationBufferMemory(chat_memory=InMemoryChatMessageHistory(messages=[HumanMessage(content='Mexican'), AIMessage(content='\n\n"El Dorado Cantina"'), HumanMessage(content='Indian'), AIMessage(content='\n\n"Maharaja\'s Palace: Fine Indian Cuisine"')]))

In [54]:
print(chain.memory.buffer)

Human: Mexican
AI: 

"El Dorado Cantina"
Human: Indian
AI: 

"Maharaja's Palace: Fine Indian Cuisine"


### Another way to reduce cost

In [55]:
from langchain.chains import ConversationChain

convo = ConversationChain(llm=OpenAI(temperature=0.7))
print(convo.prompt.template)


c:\Python312\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


In [56]:
convo.run("Who won the first cricket world cup?")

' The first cricket world cup was won by the West Indies in 1975. The tournament was held in England and was organized by the International Cricket Council (ICC). The West Indies team, also known as the Windies, defeated Australia in the final by 17 runs to claim the title. The captain of the West Indies team was Clive Lloyd and the star player was Viv Richards. This was the first time the cricket world cup was held and it marked the beginning of a global cricket tournament that is now held every four years.'

In [57]:
convo.run("What is 5+5?")

' 5+5 is equal to 10. This is a basic arithmetic equation and the answer is a fundamental mathematical concept. In mathematics, addition is the operation of combining two or more numbers to form a larger sum. In this case, we are adding two numbers, 5 and 5, to get a total of 10. This concept is used in everyday life, from counting money to calculating measurements. '

In [58]:
convo.run("who was the captian of the winning team?")

' The captain of the winning team, the West Indies, was Clive Lloyd. He was a legendary cricketer and led the team to victory in the first cricket world cup in 1975. Lloyd was known for his strong leadership skills and excellent batting abilities. He also went on to lead the West Indies team to win the second cricket world cup in 1979. After his retirement, he continued to be involved in cricket as a coach and administrator.'

In [61]:
print(convo.memory.buffer)

Human: Who won the first cricket world cup?
AI:  The first cricket world cup was won by the West Indies in 1975. The tournament was held in England and was organized by the International Cricket Council (ICC). The West Indies team, also known as the Windies, defeated Australia in the final by 17 runs to claim the title. The captain of the West Indies team was Clive Lloyd and the star player was Viv Richards. This was the first time the cricket world cup was held and it marked the beginning of a global cricket tournament that is now held every four years.
Human: What is 5+5?
AI:  5+5 is equal to 10. This is a basic arithmetic equation and the answer is a fundamental mathematical concept. In mathematics, addition is the operation of combining two or more numbers to form a larger sum. In this case, we are adding two numbers, 5 and 5, to get a total of 10. This concept is used in everyday life, from counting money to calculating measurements. 
Human: who was the captian of the winning team

# Understanding Snowflake 

In [5]:
%pip install --upgrade pip
%pip install "snowflake-snowpark-python[pandas]" snowflake-sqlalchemy
%pip install langchain openai langchain-experimental jupyter
%pip install snowflake

Note: you may need to restart the kernel to use updated packages.
  Using cached snowflake_sqlalchemy-1.5.3-py3-none-any.whl.metadata (20 kB)
Note: you may need to restart the kernel to use updated packages.


ERROR: Ignored the following versions that require a different python version: 0.10.0 Requires-Python ==3.8.*; 0.11.0 Requires-Python ==3.8.*; 0.12.0 Requires-Python ==3.8.*; 0.6.0 Requires-Python ==3.8.*; 0.7.0 Requires-Python ==3.8.*; 0.8.0 Requires-Python ==3.8.*; 0.9.0 Requires-Python ==3.8.*; 1.0.0 Requires-Python ==3.8.*; 1.1.0 Requires-Python ==3.8.*; 1.10.0 Requires-Python >=3.8, <3.12; 1.11.0 Requires-Python >=3.8, <3.12; 1.11.1 Requires-Python >=3.8, <3.12; 1.12.0 Requires-Python >=3.8, <3.12; 1.12.1 Requires-Python >=3.8, <3.12; 1.13.0 Requires-Python >=3.8, <3.12; 1.14.0 Requires-Python <3.12,>=3.8; 1.15.0 Requires-Python <3.12,>=3.8; 1.16.0 Requires-Python <3.12,>=3.8; 1.2.0 Requires-Python ==3.8.*; 1.3.0 Requires-Python ==3.8.*; 1.4.0 Requires-Python ==3.8.*; 1.5.0 Requires-Python >=3.8, <3.10; 1.5.1 Requires-Python >=3.8, <3.11; 1.6.0 Requires-Python >=3.8, <3.11; 1.6.1 Requires-Python >=3.8, <3.11; 1.7.0 Requires-Python >=3.8, <3.11; 1.8.0 Requires-Python >=3.8, <3.11; 

Note: you may need to restart the kernel to use updated packages.
INFO: pip is looking at multiple versions of snowflake to determine which version is compatible with other requirements. This could take a while.
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for snowflake: filename=snowflake-0.0.4-py3-none-any.whl size=3263 sha256=94471b7f335dd68f07fbedae76a12a921b099403f3525c8c8447be95c696d147
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\75\cd\a8\9035423587596a8207a08b3a93ada0ee941b0a0127281f5af7
Successfully built snowflake
Note: you may need to restart the kernel

ERROR: Could not install packages due to an OSError: [Errno 13] Permission denied: 'c:\\Python312\\Scripts\\snowflake'
Consider using the `--user` option or check the permissions.



In [14]:
from langchain import OpenAI, SQLDatabase
from snowflake.snowpark import Session
from langchain.chains import create_sql_query_chain

ModuleNotFoundError: No module named 'snowflake.snowpark'; 'snowflake' is not a package

In [22]:
%pip install sqlalchemy

Note: you may need to restart the kernel to use updated packages.


In [29]:
import pandas as pd
from sqlalchemy import create_engine
#from snowflake.sqlalchemy import URL
import snowflake 

df = None
with snowflake.connect(
    user="",
    account='',
    #private_key_file='/home/lou/snow-keys/rsa_key.p8',
    warehouse="",
    database="",        
    schema='',
    role='',
) as con:
    df = pd.read_sql("SELECT * FROM raw_jobs", con)
df

AttributeError: module 'snowflake' has no attribute 'connect'